In [1]:
import os
os.chdir("/Users/morizin/Documents/Code/jigsaw-competition")
from src.jigsaw.config.config import ConfigurationManager
from src.jigsaw.entity.config_entity import ModelTrainingConfig

cfg = ConfigurationManager()

config = cfg.get_model_training_config()

2025-09-30 23:12:49,332 [INFO] : common : Loading YAML file : ./config/config.yaml
2025-09-30 23:12:49,335 [INFO] : common : Loading YAML file : ./params.yaml
2025-09-30 23:12:49,338 [INFO] : common : Loading YAML file : ./schema.yaml
2025-09-30 23:12:49,340 [INFO] : common : Loading Json file : artifacts/data/status.json
2025-09-30 23:12:49,341 [INFO] : common : Successfully loaded file: artifacts/data/status.json -> 1 master keys.


In [2]:
import os
import pandas as pd
from typeguard import typechecked
from src.jigsaw import logger
from pandas.core.frame import DataFrame
from src.jigsaw.utils.common import load_csv

# @typechecked
def get_train_test_split(config: ModelTrainingConfig) -> tuple[DataFrame | None]:
    data_coll = []
    for dataset in config.schemas:

        features = dataset.features.copy()

        if dataset.target not in features:
            features.append(dataset.target)

        if config.fold >= 0:
            features.append('fold')

        for file in dataset.train:
            data = load_csv(config.indir.path / dataset.name / file)

            if all([col in data.columns for col in features]):
                data_coll.append(data[features])
            else:
                logger.error(
                    f"The dataset can't be inlcuded as it have unmatched columns names {data.columns}"
                )
    data = pd.concat(data_coll, axis=0)
    if config.fold >= 0 and config.fold <= data.fold.max():
        train_data = data.query("fold != @config.fold")
        valid_data = data.query("fold == @config.fold")
    else:
        train_data = data
        valid_data = None
    return train_data, valid_data

train_data, valid_data = get_train_test_split(config)
train_data.shape, valid_data.shape

2025-09-30 23:12:51,032 [INFO] : common : Reading artifacts/data/folded_parse_cleaned_raw/train.csv file ...
2025-09-30 23:12:51,065 [INFO] : common : Successfully read the CSV artifacts/data/folded_parse_cleaned_raw/train.csv : (1938, 10)


((1550, 10), (388, 10))

In [3]:
from src.jigsaw.components.dataset.classfier_dataset import ClassifierDataset

train_dataset = ClassifierDataset(config, train_data)

In [4]:
from src.jigsaw.components.models.classifier_model import get_deberta_model
model = get_deberta_model(config)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# import torch.nn as nn
# from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# model = AutoModelForSequenceClassification.from_pretrained(config.model_name, trust_remote_code = True)
# model.classifier = nn.Linear(model.classifier.in_features, 1)
# del model.dropout

AttributeError: 'ModelTrainingConfig' object has no attribute 'model_name'

In [17]:
config.engine.warmup_ratio

In [5]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=config.outdir.path,
    overwrite_output_dir=True,
    do_train=True,
    per_device_train_batch_size=config.engine.train_batch_size,
    gradient_accumulation_steps= config.engine.gradient_accumulation_steps,
    learning_rate=config.engine.learning_rate,
    weight_decay=config.engine.weight_decay,
    warmup_ratio= config.engine.warmup_ratio,
    num_train_epochs= config.engine.nepochs,
    report_to= 'none',
    save_strategy='no'
)

trainer = Trainer(
    model = model, 
    args = training_args,
    train_dataset=train_dataset
)

In [6]:
trainer.train()

RuntimeError: MPS backend out of memory (MPS allocated: 6.96 GiB, other allocations: 1.85 GiB, max allowed: 9.07 GiB). Tried to allocate 376.53 MiB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).